# <b>Chatbot with Retrieval-Augmented Generation (RAG) <B>#

In [17]:
pip install faiss-cpu sentence-transformers langchain openai transformers langchain-openai

In [19]:
# Import required libraries
import faiss
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import os

# Step 1: Load Knowledge Base
def load_knowledge_base(file_path):
    """Load the knowledge base from a text file."""
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    return [line.strip() for line in lines]

# Step 2: Create FAISS Index with Local Embeddings
def create_faiss_index_local(data):
    """Create a FAISS index using local embeddings."""
    print("Generating embeddings with a local model...")
    embeddings_model = SentenceTransformer('all-MiniLM-L6-v2')  # Use a local embeddings model
    embeddings = embeddings_model.encode(data)  # Generate embeddings for data

    # Create a FAISS index
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)  # L2 distance index
    index.add(embeddings)  # Add embeddings to the index

    return index, data

# Step 3: Set Up Local Model for Response Generation
def initialize_local_model():
    """Initialize a local Hugging Face transformer model for text generation."""
    print("Loading local language model for response generation...")
    generator = pipeline("text-generation", model="gpt2", max_length=200, num_return_sequences=1)
    return generator

# Step 4: RAG Pipeline Functions
def retrieve_top_match(query, index, knowledge_data):
    """Retrieve the top match from the FAISS index."""
    embeddings_model = SentenceTransformer('all-MiniLM-L6-v2')
    query_embedding = embeddings_model.encode([query])  # Generate query embedding
    distances, indices = index.search(query_embedding, 1)  # Retrieve top-1 match
    return knowledge_data[indices[0][0]], distances[0][0]

def generate_answer(context, question, generator):
    """Generate an answer using the local model."""
    prompt = f"Context: {context}\nQuestion: {question}\nAnswer:"
    response = generator(prompt, max_length=200, num_return_sequences=1)
    return response[0]["generated_text"].split("Answer:")[-1].strip()

# Main Execution
if __name__ == "__main__":
    # Step 1: Load the knowledge base
    kb_file = "knowledge_base.txt"  # Replace with your file
    knowledge_data = load_knowledge_base(kb_file)
    print("Knowledge base loaded.")

    # Step 2: Create the FAISS index
    faiss_index, knowledge_data = create_faiss_index_local(knowledge_data)
    print("FAISS index created with local embeddings.")

    # Step 3: Initialize the local language model
    generator = initialize_local_model()
    print("Local model for response generation is ready.")

    # Step 4: RAG Pipeline
    while True:
        user_query = input("\nEnter your query (type 'exit' to quit): ")
        if user_query.lower() == "exit":
            print("Goodbye!")
            break

        # Retrieve context
        context, score = retrieve_top_match(user_query, faiss_index, knowledge_data)
        print(f"\nRetrieved Context (Score: {score}):\n{context}\n")

        # Generate answer
        answer = generate_answer(context, user_query, generator)
        print(f"Answer: {answer}")


Knowledge base loaded.
Generating embeddings with a local model...
FAISS index created with local embeddings.
Loading local language model for response generation...


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...


Retrieved Context (Score: 0.42265596985816956):
The Eiffel Tower is located in Paris, France, and was completed in 1889.

Answer: According to the Bible, the Eiffel Tower was built on the ground, not on stone. In modern times, the

Enter your query (type 'exit' to quit): who is elon musk


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Retrieved Context (Score: 1.7543220520019531):
Electric vehicles (EVs) are powered by electricity stored in batteries and produce zero emissions.

Answer: "Elon musk" means, "The first ever electric vehicle (EV) is powered by electricity of two kinds, each produced by one and all batteries, and all diesel engines."
1. Electric vehicles (EVs) are first to be introduced to the market in India in October 2014, with the first vehicles expected to debut in the country by March 2015. However, the electrification of all EVs is not to begin until 2017 and has yet to be implemented.
2. The first vehicles to reach markets are the Tesla Model S, Audi E-Golf, BMW i3 and Audi A8.
3. In spite of the huge demand, the US has failed to get EVs in India and they have started selling them around the world in the wake of the December 2015 earthquake in Kazan in which 50,

Enter your query (type 'exit' to quit): who is modi


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Retrieved Context (Score: 1.7357327938079834):
The Taj Mahal in India was commissioned by Emperor Shah Jahan in memory of his wife Mumtaz Mahal.

Answer: Mohammad bin Laden has a lot of friends, he is a very respected person and is in a position to bring about some kind of changes in the world and there is nothing he can do to stop what his activities will do to its inhabitants, who are in his place of wealth. Therefore he has always been considered one of the richest men and the ultimate one. Thus, he can only help us to find more information about how he is helping the world through his activities. In particular since he has become the one who had to do much to spread prosperity, the best way we can get information on this is into his home."
In connection with this article, I had been informed that the Pakistani Army has been working with the Pakistani Government for several years to increase manpower of the forces.
The military training project was recently

Enter your query (type 